In [1]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets

Parameters of the LDO chip

In [2]:
ldo_output = 3.3  # Output voltage of the LDO (In Volts)
ldo_variance = 0.03  # Need to check datasheet for this value

Search Window

In [3]:
i_fraction_list = np.arange(0.05, 0.5, 0.05)  # fraction of current in one of the. it is symmetric, so using half window

i_load_list = np.arange(0.1, 0.55, 0.05)  # load current in mA

Function to give us the BALLAST_RESISTOR & DROPOUT_VOLTAGE for a given load current and current fraction through 1 LDO

In [4]:
def get_electrical_params(i_load, i_frac, ldo_var):

    v_diff = 2*ldo_var*ldo_output
    r_ballast = v_diff/(i_load*abs(1-2*i_frac))

    # dropout_voltage_max = r_ballast*i_load*max(i_frac, 1-i_frac)
    dropout_voltage = v_diff+0.5*(r_ballast*i_load-v_diff)
    # return r_ballast, dropout_voltage_max
    return r_ballast, dropout_voltage

In [5]:
def get_calculations(ldo_vari, iload_list=i_load_list, ifraction_list=i_fraction_list):
    
    calc_flag = False
    resistor_matrix = np.zeros((len(i_load_list), len(i_fraction_list)), dtype=float)
    # An array cause the dropout voltage is independent of the load current and depends only on the fraction of load current
    dropout_voltage_array = np.zeros((len(i_fraction_list)), dtype=float) 

    for ind1 in range(resistor_matrix.shape[0]):

        for ind2 in range(resistor_matrix.shape[1]):

            r_b, do_volt = get_electrical_params(iload_list[ind1], ifraction_list[ind2], ldo_var=ldo_vari)

            resistor_matrix[ind1, ind2] = r_b

            if not calc_flag:

                dropout_voltage_array[ind2] = do_volt

        calc_flag = True
    
    return resistor_matrix, dropout_voltage_array
    

In [6]:
def response(change):
    # Function to change the graphics
        
    res_mat, do_volt_arr = get_calculations(float_text_R.value)
        
    with g1.batch_update():
        g1.data[0].z = res_mat
        
    with g2.batch_update():
#         g2.data[0].y = []
        g2.data[0].y = do_volt_arr*1e3
        


Fill in the data for resistor matrix and for dropout voltage

In [7]:
r_mat, do_volt_array = get_calculations(ldo_variance)

In [8]:
fig1 = go.Figure()

trace1 = go.Surface(x=i_fraction_list, y=(i_load_list*1e3), z=r_mat)

fig1.add_trace(trace1)

fig1.update_layout(scene = dict(xaxis_title="Load Current Fraction-->", 
                   yaxis_title="Load Current (mA)-->", 
                   zaxis_title="Ballast Resistance (ohm)-->"), 
                   margin=dict(l=65, r=50, b=65, t=90), 
                  title="Ballast Resistor Values",
                  title_x=0.5)

fig2 = go.Figure()

trace2 = go.Scatter(x=i_fraction_list, y=(do_volt_array*1e3))

fig2.add_trace(trace2)

fig2.update_layout(title='Dropout Voltage Behaviour',
                   title_x=0.5,
                  xaxis_title='Load Current Fraction -->',
                  yaxis_title='Dropout Voltage (mV)-->')

g1 = go.FigureWidget(fig1)
g2 = go.FigureWidget(fig2)

float_text_R = widgets.BoundedFloatText(
    value=ldo_variance,
    min=0.01,
    max=0.05,
    step=0.01,
    description='Output Variance: ',
    disabled=False
)

float_text_R.observe(response, names="value")

# container = widgets.VBox([float_text_R, fig1, fig2])

# display(widgets.VBox([fig1, fig2]))
container1 = widgets.HBox([float_text_R])
display(widgets.VBox([container1, g1, g2]))
